In [1]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [7]:
%load_ext autoreload
%autoreload 2

In [13]:
from modules.probabilities import Probabilities


In [4]:
# Derive Probabilties for transition
np.random.seed(14)
prob = Probabilities()

prob.getProbabilities(300).head()


,Trip,Length,Price,p
0,y,0,10,0.000262858
1,y,0,20,0.000248595
2,y,4,10,0.000242638
3,y,4,20,0.000229473
4,y,8,10,0.000192089


In [27]:
from modules.app import App
a = App()
ss = a.constructStates()
for s in ss:
    print(s.__str__())
    for d in a.constructDecisions(s):
        print(d.__str__())
    
    print()

State t=1[(2,25, 10,5,5) - False - (0.000000, 0.000000)]
Decision [(0.000000,0.000000,0)]

State t=1[(0,25, 10,5,5) - False - (0.000000, 0.000000)]
Decision [(0.000000,0.000000,0)]
Decision [(0.000000,0.000000,1)]
Decision [(0.500000,0.000000,0)]
Decision [(1.000000,0.000000,0)]
Decision [(1.500000,0.000000,0)]
Decision [(2.000000,0.000000,0)]
Decision [(2.500000,0.000000,0)]
Decision [(0.000000,0.500000,0)]
Decision [(0.000000,1.000000,0)]
Decision [(0.000000,1.500000,0)]
Decision [(0.000000,2.000000,0)]
Decision [(0.000000,2.500000,0)]

State t=1[(0,25, 0,5,5) - False - (0.000000, 0.000000)]
Decision [(0.000000,0.000000,0)]
Decision [(0.500000,0.000000,0)]
Decision [(1.000000,0.000000,0)]
Decision [(1.500000,0.000000,0)]
Decision [(2.000000,0.000000,0)]
Decision [(2.500000,0.000000,0)]
Decision [(0.000000,0.500000,0)]
Decision [(0.000000,1.000000,0)]
Decision [(0.000000,1.500000,0)]
Decision [(0.000000,2.000000,0)]
Decision [(0.000000,2.500000,0)]

